In [1]:

import os
fname_prefix = "/data/users/team2_capstone/code-style-probing/"
os.environ["CUDA_VISIBLE_DEVICES"] = '4'
import numpy as np
#import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import load_from_disk, load_metric
#import datasets
import pickle
#from utils.helper import read_py150k_code, read_file_to_string
import regex as re

In [2]:

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
#model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')
train_codet5_dataset = load_from_disk(fname_prefix + 'datasets/codet5_train_uncommented_combined_padded.hf')
test_codet5_dataset = load_from_disk(fname_prefix + 'datasets/codet5_test_uncommented_combined_padded.hf')

"""def replaceNegative(label):
    #print (label['labels'])

    label['labels'] = [label['labels']]
    return label

train_codet5_dataset = train_codet5_dataset.map(replaceNegative, batched = False)
test_codet5_dataset = test_codet5_dataset.map(replaceNegative, batched = False)"""

loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/12dd1d7e87ade1728f8382f3b875a47cdddb88bc50797ecc506957411661a39a.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/c2ada6c76bb6a90c2330323775cb4853dd2b0cfee29d6b2c5ecb419c5874b488.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/354ae288bdee65437fa8eedecf9a2770001b97bac23d7fc5a04badae8da42346.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/f420b33feb234c

"def replaceNegative(label):\n    #print (label['labels'])\n\n    label['labels'] = [label['labels']]\n    return label\n\ntrain_codet5_dataset = train_codet5_dataset.map(replaceNegative, batched = False)\ntest_codet5_dataset = test_codet5_dataset.map(replaceNegative, batched = False)"

In [3]:
model_checkpoint = "Salesforce/codet5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
eval_dataset = test_codet5_dataset
eval_dataset = eval_dataset.train_test_split(test_size = 0.08)
print (eval_dataset)

loading configuration file https://huggingface.co/Salesforce/codet5-small/resolve/main/config.json from cache at /soe/ksmunson/.cache/huggingface/transformers/ef13e715cbf36adda46c74774e8032ab573cfbb2ebe59748c9fc72b7cf67e418.96d28e790b8c3d3e3be663606a66e0793a173c78e745e3603be4c0f878319099
Model config T5Config {
  "_name_or_path": "Salesforce/codet5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
 

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'line_count', 'comment_count', 'comment_total_len', 'comment_avg_len', 'comment_density', 'id_total', 'lower_case', 'id_total_var', 'lower_case_var', 'snake_case_ratio', 'snake_case_var_ratio', 'snake_case_class_ratio', 'snake_case_method_ratio', 'lower_camel_case_ratio', 'lower_camel_case_var_ratio', 'lower_camel_case_class_ratio', 'lower_camel_case_method_ratio', 'upper_camel_case_ratio', 'upper_camel_case_var_ratio', 'upper_camel_case_class_ratio', 'upper_camel_case_method_ratio', 'lower_case_ratio', 'lower_case_var_ratio', 'lower_case_class_ratio', 'lower_case_method_ratio', 'upper_case_ratio', 'upper_case_var_ratio', 'upper_case_class_ratio', 'upper_case_method_ratio', 'other_case_ratio', 'other_case_var_ratio', 'other_case_class_ratio', 'other_case_method_ratio', 'func_decorators_avg', 'func_async_ratio', 'class_parents_avg', 'class_decorators_avg', 'ds_density', 'ds_char_len_avg', 'ds_word_len_av

In [24]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
from transformers import AutoConfig

#config = AutoConfig.from_pretrained(fname_prefix + "/seq2seq_results/no_outlier_codet5small/checkpoint-51000",    min_length = 20, max_length = 512, )
model =AutoModelForSeq2SeqLM.from_pretrained(fname_prefix + "/seq2seq_results/no_outlier_codet5small/checkpoint-51000")
model.config.max_length = 512
#fname_prefix + /seq2seq_results/no_outlier_codet5small/checkpoint-50500
args = Seq2SeqTrainingArguments(
    #f"{model_name}-finetuned-xsum",
    output_dir= fname_prefix + "/seq2seq_results/no_outlier_codet5small",
    evaluation_strategy="steps",
    eval_steps=20000,
    eval_accumulation_steps=200,
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=30,
    num_train_epochs=10,
    predict_with_generate=True,
    push_to_hub=False,

)

#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

loading configuration file /data/users/team2_capstone/code-style-probing//seq2seq_results/no_outlier_codet5small/checkpoint-51000/config.json
Model config T5Config {
  "_name_or_path": "/data/users/team2_capstone/code-style-probing//seq2seq_results/no_outlier_codet5small/checkpoint-51000",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_b

In [25]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_codet5_dataset ,
    eval_dataset=test_codet5_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    
)
model.generate()
#
#trainer.train()

tensor([[    0,     1,     7,     5,    19, 13640,    19,  4757,    19,  3074,
          5790,   203,     7,   300,    14,    17, 14848,    30,  7718,    17,
            28,   300,    14,    17,   203,   203,     7, 25417,   261,    71,
            13, 12461,    16,   478,  1074, 19861, 11473,    18,  8660, 20317,
           348, 18739, 18465,   203,     7,  4826, 14989,  8735,    18,   203,
           203,     7,  8505,  4027,   471,   999,   316,  1084,   471,  3112,
         10138,    16,   598,   578,  2887,   203,     7, 11544,    16,   854,
         15498,  2112,   716,   326,  3751,  4636,   854,  5100,    30,   203,
           203,     7,   404,    18,  8505, 15326,   434,  1084,   981,  1297,
         15096,   326,  5721, 22245, 11690,    16,   333,   203,     7,   666,
           434,  4636,   471,   326,  3751,  1015,   830,    69,  4417,    18,
           203,   203,     7,   576,    18,  8505, 15326,   316,  3112,   646,
          1297,   283, 11776,   311,   326,  5721, 2

In [26]:
eval_preds = trainer.predict(eval_dataset['test']) #, min_length=40, max_length = 512)

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: lower_case_method_ratio, other_case_var, lower_case_var, comprehensions_avg, other_case_ratio, lower_camel_case_class, ds_char_len_avg, forks, lower_camel_case_ratio, ds_count, lower_camel_case_class_ratio, func_async_ratio, lower_case, upper_camel_case_method_ratio, other_case_method_ratio, upper_case_method_ratio, func_async_count, class_decorators_count, file, func_count, comment_count, lower_case_ratio, content, func_decorators_count, upper_camel_case_class_ratio, generators, upper_camel_case_ratio, lambda, internal_method, upper_case_class, ds_word_len_total, lower_case_var_ratio, lower_case_class_ratio, lower_case_method, upper_camel_case_var, Unnamed: 0.1, comment_total_len, id_total, parse_error, other_case_method, upper_case_var_ratio, comprehensions, id_total_method, ds_word_len_avg, upper_case_ratio, overridden_method, snake_case_class_rati

In [30]:
predictions, labels, report  = eval_preds
print (predictions.shape)
idx = 30
decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
print ("pred:", decoded_preds[idx]) 
print ("label:", decode_labels[idx])

(3240, 512)
pred: import datetime
try:
    from urlparse import urlparse
except ImportError:
    from urllib.parse import urlparse
from xml.dom import minidom

from django.conf import settings
from django.test.client import RequestFactory
from django.template import Template, Context
from django.core.urlresolvers import reverse
from django.test import TestCase
from django.utils.unittest import skipIf

from paypal.pro.models import PayPalNVP

from billing import get_gateway, get_integration, CreditCard
from billing.signals import *
from billing.gateway import CardNotSupported
from billing.utils.credit_card import Visa

RF = RequestFactory()

request = RF.get('/', REMOTE_ADDR='192.168.1.1')

fake_options = {
   'request': request,
    'email': 'testuser@fakedomain.com',
    'billing_address': {
        'name': 'PayPal User',
        'address1': 'Street 1',
        'city': 'Mountain View',
       'state': 'CA',
        'country': 'US',
        'zip': '94043',
    }
}


@skipIf(not setting